In [51]:
import httpx
endpoint = "http://127.0.0.1:8000/"
# prompt = "a phot of TOK who is a architecture explaining building plans to a contractor"
res = httpx.get(endpoint)
res.status_code, res.json(), res.headers.get('Retry-After')

(429, {'detail': 'Too many requests, try again soon. '}, '1')

In [32]:
r

NameError: name 'r' is not defined

In [30]:
res.headers.get("X-Forwarded-For")